In [1]:
import robosuite as suite
from robosuite.wrappers.gym_wrapper import GymWrapper
import gymnasium as gym
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.monitor import Monitor
import time
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.callbacks import BaseCallback
import numpy as np


[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /home/ja/PycharmProjects/robosuite/robosuite/robosuite/scripts/setup_macros.py (macros.py:55)


In [2]:
import os
filename = 'tmp/gym/multiinput_256'
cwd = os.getcwd()
new_folder = os.path.join(cwd, filename)
os.makedirs(new_folder, exist_ok=True)

In [3]:
# create environment instance
env = GymWrapper(suite.make(
    env_name="Lift", # try with other tasks like "Stack" and "Door"
    robots="UR5e",  # try with other robots like "Sawyer" and "Jaco"
    has_renderer=True,
    has_offscreen_renderer=True,
    use_object_obs=False,                   # don't provide object observations to agent
    use_camera_obs=True,
    camera_names="robot0_eye_in_hand",      # use "agentview" camera for observations
    camera_heights=84,                      # image height
    camera_widths=84,                       # image width
    reward_shaping=True,                    # use a dense reward signal for learning
    horizon = 500,
    control_freq=20,                        # control should happen fast enough so that simulation looks smooth
))
env = Monitor(env, filename)
# ), ['robot0_eye_in_hand_image'])

In [4]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq: (int)
    :param log_dir: (str) Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: (int)
    """

    def __init__(self, check_freq: int, log_dir: str, verbose=1):
        super().__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, "best_model")
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:

            # Retrieve training reward
            x, y = ts2xy(load_results(self.log_dir), "timesteps")
            if len(x) > 0:
                # Mean training reward over the last 100 episodes
                mean_reward = np.mean(y[-100:])
                if self.verbose > 0:
                    print(f"Num timesteps: {self.num_timesteps}")
                    print(
                        f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per episode: {mean_reward:.2f}"
                    )

                # New best model, you could save the agent here
                if mean_reward > self.best_mean_reward:
                    self.best_mean_reward = mean_reward
                    # Example for saving best model
                    if self.verbose > 0:
                        print(f"Saving new best model to {self.save_path}.zip")
                    self.model.save(self.save_path)

        return True

In [5]:
env.keys

/home/ja/anaconda3/envs/robosuite/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.keys to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.keys` for environment variables or `env.get_wrapper_attr('keys')` that will search the reminding wrappers.
  logger.warn(


['robot0_joint_pos_cos',
 'robot0_joint_pos_sin',
 'robot0_joint_vel',
 'robot0_eef_pos',
 'robot0_eef_quat',
 'robot0_gripper_qpos',
 'robot0_gripper_qvel',
 'robot0_eye_in_hand_image',
 'robot0_proprio-state',
 'robot0_eye_in_hand_image',
 'robot0_proprio-state']

In [6]:
from stable_baselines3 import PPO
policy_kwargs = dict(
    net_arch=[256, 256]
)
model = PPO("MultiInputPolicy", env, verbose=1, batch_size=256, policy_kwargs=policy_kwargs)
callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=filename)
model.learn(total_timesteps=5e5, progress_bar=True, log_interval=10, callback=callback)

Using cuda device
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


Output()

Num timesteps: 1000

Best mean reward: -inf - Last mean reward per episode: 0.76

Saving new best model to tmp/gym/multiinput_256/best_model.zip

...

Best mean reward: 107.52 - Last mean reward per episode: 76.14

Num timesteps: 500000

Best mean reward: 107.52 - Last mean reward per episode: 75.93

Num timesteps: 501000

Best mean reward: 107.52 - Last mean reward per episode: 73.17

In [ ]:
model.save(filename)

In [8]:
obs = env.reset()[0]
action_list = []
for i in range(500):
    action, state = model.predict(obs)
    action_list.append(action)
    obs, reward, done,done, info = env.step(action)
    env.render()
    # time.sleep(0.2)
    if done:
        break
env.close()

QObject::moveToThread: Current thread (0xe9abfb0) is not the object's thread (0xcd03960).
Cannot move to target thread (0xe9abfb0)

QObject::moveToThread: Current thread (0xe9abfb0) is not the object's thread (0xcd03960).
Cannot move to target thread (0xe9abfb0)

QObject::moveToThread: Current thread (0xe9abfb0) is not the object's thread (0xcd03960).
Cannot move to target thread (0xe9abfb0)

QObject::moveToThread: Current thread (0xe9abfb0) is not the object's thread (0xcd03960).
Cannot move to target thread (0xe9abfb0)

QObject::moveToThread: Current thread (0xe9abfb0) is not the object's thread (0xcd03960).
Cannot move to target thread (0xe9abfb0)

QObject::moveToThread: Current thread (0xe9abfb0) is not the object's thread (0xcd03960).
Cannot move to target thread (0xe9abfb0)

QObject::moveToThread: Current thread (0xe9abfb0) is not the object's thread (0xcd03960).
Cannot move to target thread (0xe9abfb0)

QObject::moveToThread: Current thread (0xe9abfb0) is not the object's thread

ValueError: I/O operation on closed file.

In [ ]:
from stable_baselines3 import PPO
model = PPO.load('trained_model', env=env)
model.learn(total_timesteps=100000, progress_bar=True, log_interval=10)
model.save('trained_model')


In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
import robosuite.macros as macros
macros.IMAGE_CONVENTION = "opencv"
env = GymWrapper(suite.make(
    env_name="Lift", # try with other tasks like "Stack" and "Door"
    robots="UR5e",  # try with other robots like "Sawyer" and "Jaco"
    has_renderer=False,
    ignore_done = True,
    has_offscreen_renderer=True,
    use_object_obs=True,                   # don't provide object observations to agent
    use_camera_obs=True,
    camera_names="agentview",      # use "agentview" camera for observations
    camera_heights=512,                      # image height
    camera_widths=512,                       # image width
    reward_shaping=True,                    # use a dense reward signal for learning
    horizon = 500,
    control_freq=20,                        # control should happen fast enough so that simulation looks smooth
))

In [ ]:
import imageio

writer = imageio.get_writer(f'{filename}/video.mp4', fps=20)
obs = env.reset()[0]
for i,v in enumerate(action_list):
    obs, reward, done,done, info = env.step(v)
    frame = obs["agentview_image"]
    writer.append_data(frame)
    print("Saving frame #{}".format(i))
    if done:
        break
writer.close()
env.close()